# Clusterization

## 1. Texts Segmentation

In [2]:
from clusterization import text_segmentation
new = text_segmentation("../../../Texts/text2_3.txt")
new[:5]

[[(0,
   ' А когда речь заходит о покупке авто на вторичном рынке, то ещё и опасное.'),
  (0, ' Ведь подержанный автомобиль – это всегда «кот в мешке».'),
  (1,
   ' Вопреки описанию и виду, можно «напороться» и на технически неисправную машину, и на битую-перебитую, а порой и на «криминальную».'),
  (1,
   ' Однако добрую часть вариантов можно отсеять ещё на этапе просмотра объявлений, нужно только быть внимательным.')],
 [[(0, ''),
   (1, ' Грамотно составляйте запрос поиска\n'),
   (1,
    'Для того чтобы не тратить время и силы, нужно максимально точно определиться с тем, что, собственно, необходимо приобрести и сколько на покупку имеется денег.')]],
 [(0, ' Если нужен седан, значит, выбираете только седан.'),
  (0,
   ' Если нужна механическая коробка, в условиях поиска ставите «только МКП».'),
  (1,
   ' В идеале, конечно, к моменту составления запроса нужно четко представлять, автомобиль какой марки и модели предполагается к покупке.'),
  (0,
   ' Нужен Рено Логан, значит, смотр

## 2. Experiments With Semantic

In [7]:
import os
import numpy as np
import time
import pickle
from scipy.linalg import block_diag


In [8]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('../../model.bin', binary=True) 
model.init_sims(replace=True)

/home/ased/anaconda3/lib/python3.5/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
import sys
sys.path.append('../../')
import sem_analysis as sa

In [10]:
newest = list()
ind =0
for i in new:
    _ = list()
    for j in i:
        if type(j) == type(tuple()):
            item = (j[0], sa.tag_ud(j[1]))
        else:
            item = list()
            for k in j:
                item.append((k[0], sa.tag_ud(k[1])))
        _.append(item)
    ind+=1
    newest.append(_)

In [11]:
full_list = list()
for i in newest:
    cur_list = [j for j in i]
    if len(cur_list) >0 and type(cur_list[0]) != type(tuple()):
        _ = list()
        for j in cur_list:
            _ = _ + j
        cur_list = _
    full_list = full_list + (cur_list)

In [12]:
sentences = list()
for i in new:
    cur_list = [j for j in i]
    if len(cur_list) >0 and type(cur_list[0]) != type(tuple()):
        _ = list()
        for j in cur_list:
            _ = _ + j
        cur_list = _
    sentences = sentences + (cur_list)
len(full_list), len(sentences), len(full_list_centers)

NameError: name 'full_list_centers' is not defined

In [13]:
def get_center(sentence, model = model):
    s = np.zeros(300)
    k = 0
    for i in sentence:
        if model.__contains__(i):
            s = s + model[i]
            k+=1
    return s/k
full_list_centers = [(i[0], get_center(i[1])) for i in full_list]

/home/ased/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: RuntimeWarning: invalid value encountered in true_divide


In [128]:
c_prev, c_next = None, None
eps = 0.9
res = list()
for _, j in enumerate(full_list_centers):
    if j[0] == 1:
        c_next = (_,j)
        break
for ind, i in enumerate(full_list_centers):
    if i[0] == 1:
        c_prev = (ind, i)
        res.append((ind, 0.))
        c_next = None
        for _, j in enumerate(full_list_centers[ind+1:]):
            if j[0] == 1:
                c_next = (ind+1+_,j)
                break
    else:
        if not c_prev is None:
            d_prev = distance(c_prev[1][1], i[1])
        else:
            d_prev = 1000
        if not c_next is None:
            d_next = distance(c_next[1][1], i[1])
        else:
            d_next = 1000
        if d_next > eps and d_prev > eps:
            res.append((None, None))
        elif d_next > d_prev: 
            res.append((c_prev[0], d_prev))
        else:
            res.append((c_next[0], d_next))

In [129]:
res

[(3, 0.5557766437264194),
 (3, 0.45316148751001484),
 (3, 0.4431509170213725),
 (3, 0.0),
 (4, 0.0),
 (4, 0.505034185264523),
 (6, 0.0),
 (None, None),
 (None, None),
 (13, 0.6630644594962454),
 (13, 0.3593313600803026),
 (13, 0.3850419835407805),
 (13, 0.41026673063296926),
 (13, 0.0),
 (14, 0.0),
 (14, 0.39543659309601875),
 (16, 0.0),
 (16, 0.48101715038759113),
 (19, 0.5199172571194135),
 (19, 0.0),
 (20, 0.0),
 (20, 0.7756189113392943),
 (24, 0.6006834359666579),
 (24, 0.5101977652589856),
 (24, 0.0),
 (25, 0.0),
 (29, 0.517473692965699),
 (29, 0.49608365034432),
 (29, 0.504080929013464),
 (29, 0.0),
 (None, None),
 (29, 0.6566085899597963),
 (29, 0.5254180208701696),
 (33, 0.0),
 (34, 0.0),
 (36, 0.3795443831523851),
 (36, 0.0),
 (36, 0.40118072436003616),
 (None, None),
 (39, 0.0),
 (39, 0.6820321889083104),
 (39, 0.4067230978432624),
 (39, 0.42470744525888715),
 (39, 0.539320296837183),
 (39, 0.5554899668399986),
 (39, 0.6903675314220672),
 (47, nan),
 (47, 0.0),
 (49, 0.537722

In [154]:
prev = None
cur_text = ''
cur_item = None
list_texts = list()
TT = list()
for ind, i in enumerate(res):
    if i[0] == prev:
        cur_text = cur_text + sentences[ind][1]
        cur_item = cur_item + full_list_centers[ind][1]
        k += 1
    else:
        if not i[0] is None:
            prev = i[0]
            if not cur_item is None:
                print(cur_text, '\n')
                list_texts.append(cur_text)
                TT.append(cur_item / k)
            cur_text = sentences[ind][1]
            cur_item = full_list_centers[ind][1]
            k = 1
print(cur_text)
list_texts.append(cur_text)
TT.append(cur_item / k)

﻿Как выбрать автомобиль с пробегом.4 простых действия при покупке авто.Существует великое множество статей о том, как купить автомобиль с пробегом и при этом не «попасть». В данной статье я попытаюсь предоставить вам примерный алгоритм действий, которые необходимо будет провести при этом мероприятии, а так же попытаюсь донести кое-какие основы "этикета покупателя", если вообще таковой имеется, дабы не довести продавца "до ручки" глупыми вопросами и действиями=)
 

Итак, после того как вы определились, какую марку и модель вы хотите приобрести, начинаем поиск. Проще всего в 21-м веке воспользоваться услугами интернет ресурсов. 

 Отмечу, что машину стоит искать на авторитетных сайтах таких как: auto. 

ru и прочие.Вторым пунктом, конечно же, телефонный звонок продавцу и несколько простых вопросов, задав которые вы существенно сэкономите своё время. Поинтересоваться, собственник ли он данного авто или его представитель, имеет ли он право продавать данный автомобиль. Является ли авто в «з

In [156]:
D = [(np.linalg.norm(i - TT[ind+1]), list_texts[ind], list_texts[ind+1]) for ind, i in enumerate(TT[:-1])]

In [157]:
D

[(0.32089609031221966,
  '\ufeffКак выбрать автомобиль с пробегом.4 простых действия при покупке авто.Существует великое множество статей о том, как купить автомобиль с пробегом и при этом не «попасть». В данной статье я попытаюсь предоставить вам примерный алгоритм действий, которые необходимо будет провести при этом мероприятии, а так же попытаюсь донести кое-какие основы "этикета покупателя", если вообще таковой имеется, дабы не довести продавца "до ручки" глупыми вопросами и действиями=)\n',
  'Итак, после того как вы определились, какую марку и модель вы хотите приобрести, начинаем поиск. Проще всего в 21-м веке воспользоваться услугами интернет ресурсов.'),
 (0.4355423316840282,
  'Итак, после того как вы определились, какую марку и модель вы хотите приобрести, начинаем поиск. Проще всего в 21-м веке воспользоваться услугами интернет ресурсов.',
  ' Отмечу, что машину стоит искать на авторитетных сайтах таких как: auto.'),
 (0.4014128295567822,
  ' Отмечу, что машину стоит искать